## 导入模块及定义函数

In [86]:
import re
import time
import pathlib
from lxml import etree
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_books_info(booklist_md):
    """读取Markdown文件的书单内容转为Dataframe"""
    books = {'分类': [], '书名': [], '说明': []}
    with open(booklist_md, encoding='utf-8') as f:
        for line in f:
            if line.startswith('### '):
                category = line.strip('# \n').split()[-1]
            elif '《' in line:
                books['书名'].append(line.strip())
                books['分类'].append(category)
            elif line.startswith('> '):
                books['说明'].append(line.strip('> \n'))

    # for k, v in books.items():
    #     print(k, len(v))

    df_books = pd.DataFrame(books)
    df_split = df_books['书名'].str.split('#', expand=True).rename(columns={0: '书名', 1: '状态'})
    df_split['书名'] = df_split['书名'].str.extract(r'《(.*?)》')
    df_books = df_books.drop(['书名'], axis=1).join(df_split)
    df_books = df_books[['分类', '书名', '状态', '说明']]
    
    return df_books

    
def get_book_url(book_name, driver, start=True):
    """根据书名获取豆瓣链接"""
    # driver.get('https://book.douban.com')
    driver.find_element(by=By.ID, value='inp-query').clear()
    driver.find_element(by=By.ID, value='inp-query').send_keys(book_name + '\n')
    
    # 显式等待
    WebDriverWait(driver, 5, 0.5).until(EC.visibility_of_element_located((By.CLASS_NAME, "root")))
    time.sleep(0.5)

    with open('test2.html', 'w', encoding='utf-8') as f:
        f.write(driver.page_source)
        
    html = etree.HTML(driver.page_source)   
    blocks = html.xpath("//div[@class='item-root']")    
    books = []
    for block in blocks:
        book = {}
        book['书名'] = block.xpath(".//a[@class='title-text']")[0].text
        book['url'] = block.xpath(".//a[@class='title-text']")[0].xpath("./@href")[0]
        try:
            evalue_num = re.search(r'\d+', block.xpath(".//span[@class='pl']")[0].text)
            book['评价人数'] = int(evalue_num.group()) if evalue_num else 0
        except Exception as e:
            pass
        try:
            book['状态'] = block.xpath(".//div[@class='status-text']")[0].text
        except Exception as e:
            book['状态'] = '无'
        books.append(book)      
    
    # 使用selenium方法获取节点信息（速度太慢）
    # blocks = driver.find_elements(by=By.CLASS_NAME, value='item-root')
    # books = []
    # for block in blocks:
    #     book = {}
    #     book['书名'] = block.find_element(by=By.CLASS_NAME, value='title-text').text
    #     book['url'] = block.find_element(by=By.CLASS_NAME, value='title-text').get_attribute('href')
    #     try:
    #         evalue_num = re.search(r'\d+', block.find_element(by=By.CLASS_NAME, value='pl').text)
    #         book['评价人数'] = int(evalue_num.group()) if evalue_num else 0
    #     except Exception as e:
    #         continue
    #     try:
    #         book['状态'] = block.find_element(by=By.CLASS_NAME, value='status-text').text
    #     except Exception as e:
    #         book['状态'] = '无'
    #     books.append(book)

    # 优先提取已标记过（读过、想读、在读）和评价人数最多的书的链接，且书名应包含搜索词前2个字
    try:
        keyword = book_name[:2]
        df_books = pd.DataFrame(books)
        if start:
            df_books = df_books[df_books['书名'].str.startswith(keyword)].sort_values(by=['状态', '评价人数'], ascending=[True, False]).reset_index(drop=True)
        else:
            df_books = df_books[df_books['书名'].str.contains(keyword)].sort_values(by=['状态', '评价人数'], ascending=[True, False]).reset_index(drop=True)
        book_url = df_books['url'][0]
        name = df_books['书名'][0]
        print(f"已获取《{book_name}》的豆瓣链接：[{name}]({book_url})……")
    except Exception as e:
        book_url = None
        name = None
        print(f"未能获取《{book_name}》的豆瓣链接！")
        
    return book_url, name


def add_book_to_list(driver, book_name, book_url):
    """将书本添加到豆瓣书单"""
    driver.find_element(by=By.CLASS_NAME, value='doulist-add-subject-btn').click()
    driver.find_element(by=By.ID, value='url-input').send_keys(book_url)
    driver.find_element(by=By.CLASS_NAME, value='url-submit').click()
    time.sleep(0.5)
    driver.find_element(by=By.CLASS_NAME, value='doulist_submit').click()
    # print(f"已添加《{book_name}》（{book_url}）到书单……")
    time.sleep(3)
    
    
def creat_book_list(driver, book_list_title, book_list_intro, all_book_lists_url):
    """创建书单"""
    driver.get(all_book_lists_url)
    book_list_window = driver.current_window_handle
    driver.find_element(by=By.XPATH, value="//a[@class='lnk-create-doulist ']").click()
    all_windows = driver.window_handles
    for window in all_windows:
        if window != book_list_window:
            driver.switch_to.window(window)
            driver.find_element(by=By.XPATH, value="//input[@name='dl_title']").send_keys(book_list_title)
            driver.find_element(by=By.XPATH, value="//textarea[@name='dl_about']").send_keys(book_list_intro)
            driver.find_element(by=By.XPATH, value="//input[@name='dl_submit']").click()
    time.sleep(1)
    book_list_url = driver.current_url
    # driver.close()
    # driver.switch_to.window(book_list_window)
    return book_list_url


def add_books_to_list(driver, book_list_url, book_names, book_urls):
    """添加所有缺失书本到书单"""
    # 打开书单网址
    driver.get(book_list_url)
    tree = etree.HTML(driver.page_source)
    existing_urls = tree.xpath("//div[@class='title']/a/@href")

    # 添加缺失书本到书单，注意让chrome页面在前台显示
    count = 0
    for book_name, book_url in zip(book_names, book_urls):
        if book_url not in existing_urls:
            try:
                add_book_to_list(driver, book_name, book_url)
                count += 1
            except Exception as e:
                print(f"添加《{book_name}》（{book_url}）到书单失败……")
                # print(e)
                continue
    return existing_urls, count

In [2]:
# # 调试
# with open('test2.html', encoding='utf-8') as f:
#     source = f.read()
    
# html = etree.HTML(source)   
# blocks = html.xpath(".//div[@class='item-root']")    
# books = []
# for block in blocks:
#     book = {}
#     book['书名'] = block.xpath(".//a[@class='title-text']")[0].text
#     book['url'] = block.xpath(".//a[@class='title-text']")[0].xpath("./@href")[0]
#     try:
#         evalue_num = re.search(r'\d+', block.xpath(".//span[@class='pl']")[0].text)
#         book['评价人数'] = int(evalue_num.group()) if evalue_num else 0
#     except Exception as e:
#         continue
#     try:
#         book['状态'] = block.xpath(".//div[@class='status-text']")[0].text
#     except Exception as e:
#         book['状态'] = '无'
#     books.append(book)    
    
# books  

## 启动Chrome浏览器

In [87]:
# 在windows命令行窗口打开Chrome，完成网站登录等手动操作
# chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\QMDownload\Python Programming\Python_Work\Web Spider\my_douban_books\my_douban_data\AutomationProfile"

options = Options()
options.page_load_strategy = 'eager'    # 快速加载
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = Service(executable_path=r'C:\Python37\Scripts\chromedriver.exe')
driver = webdriver.Chrome(service=chrome_driver, options=options)
# driver.implicitly_wait(5)


url = 'https://book.douban.com'
driver.get(url)
print(driver.title)

豆瓣读书


In [14]:
# # 测试
# book_name = '混乱： 如何成为失控时代的掌控者'
# book_url = get_book_url(book_name, driver)
# book_url

已获取《混乱： 如何成为失控时代的掌控者》的豆瓣链接：[混乱 : 如何成为失控时代的掌控者](https://book.douban.com/subject/27174024/)……


('https://book.douban.com/subject/27174024/', '混乱 : 如何成为失控时代的掌控者')

## 获取书单中的书在豆瓣的链接

In [4]:
# 首次读取md文档内容
data_folder = pathlib.Path(r'./my_douban_data')
booklist_md = data_folder / '用200本书构建知识体系.md'
df_books = get_books_info(booklist_md)
df_books

,分类,书名,状态,说明
0,历史,大历史： 虚无与万物之间,想读,（美）大卫·克里斯蒂安、（美）辛西娅·斯托克斯·布朗、（美）克雷格·本杰明著，刘耀辉译，北京...
1,历史,枪炮、病菌与钢铁： 人类社会的命运,已读,（美）贾雷德·戴蒙德著，谢延光译，上海译文出版社，2016.7
2,历史,崩溃： 社会如何选择成败兴亡,None,（美）贾雷德·戴蒙德著，江滢、叶臻译，上海译文出版社，2018.3
3,历史,贸易打造的世界： 1400年至今的社会、文化与世界经济,None,（美）彭慕兰、（美）史蒂夫·托皮克著，黄中宪、吴莉苇译，上海人民出版社，2018.2
4,历史,伯罗奔尼撒战争史,None,（古希腊）修昔底德著，谢德风译，商务印书馆，2011.7
...,...,...,...,...
195,文学,小说课,已读,许荣哲著，中信出版社，2016.8
196,文学,文章自在,已读,张大春著，广西师范大学出版社，2017.1
197,文学,〈华尔街日报〉是如何讲故事的,在读,（美）威廉·E.布隆代尔著，徐扬译，华夏出版社，2006.1
198,文学,新新新闻主义： 美国顶尖非虚构作家写作技巧访谈录,None,（美）罗伯特·博因顿著，刘蒙之译，北京师范大学出版社，2018.2


In [41]:
# 首次从头开始查询链接
results = [get_book_url(book_name, driver) for book_name in df_books['书名']] 

df_books['豆瓣链接'] = [i[0] for i in results]
df_books['备注'] = [i[1] for i in results]
df_books.to_excel(data_folder / '用200本书构建知识体系_tmp.xlsx', index=False)
df_books

已获取《大历史： 虚无与万物之间》的豆瓣链接：[大历史 : 虚无与万物之间](https://book.douban.com/subject/26827546/)……
已获取《枪炮、病菌与钢铁： 人类社会的命运》的豆瓣链接：[枪炮、病菌与钢铁 : 人类社会的命运](https://book.douban.com/subject/1813841/)……
已获取《崩溃： 社会如何选择成败兴亡》的豆瓣链接：[崩溃 : 社会如何选择成败兴亡](https://book.douban.com/subject/27609290/)……
已获取《贸易打造的世界： 1400年至今的社会、文化与世界经济》的豆瓣链接：[贸易打造的世界 : 1400年至今的社会、文化与世界经济](https://book.douban.com/subject/27155983/)……
已获取《伯罗奔尼撒战争史》的豆瓣链接：[伯罗奔尼撒战争史（上下）](https://book.douban.com/subject/6893286/)……
已获取《罗马帝国的陨落： 一部新的历史》的豆瓣链接：[罗马帝国的陨落 : 一部新的历史](https://book.douban.com/subject/26905142/)……
已获取《文明史： 人类五千年文明的传承与交流》的豆瓣链接：[文明史 : 人类五千年文明的传承与交流](https://book.douban.com/subject/25845900/)……
已获取《西方的兴起： 人类共同体史》的豆瓣链接：[西方的兴起 : 人类共同体史](https://book.douban.com/subject/27070163/)……
已获取《美国政治传统及其缔造者》的豆瓣链接：[美国政治传统及其缔造者](https://book.douban.com/subject/4729534/)……
已获取《现代世界的诞生》的豆瓣链接：[现代世界的诞生](https://book.douban.com/subject/24699353/)……
已获取《论美国的民主》的豆瓣链接：[论美国的民主](https://book.douban.com/subject/1041385/)……
已获取《旧制度与大革命》的豆瓣链接：[旧制度与大革命](https://book.douban.

,分类,书名,状态,说明,豆瓣链接,备注
0,历史,大历史： 虚无与万物之间,想读,（美）大卫·克里斯蒂安、（美）辛西娅·斯托克斯·布朗、（美）克雷格·本杰明著，刘耀辉译，北京...,https://book.douban.com/subject/26827546/,大历史 : 虚无与万物之间
1,历史,枪炮、病菌与钢铁： 人类社会的命运,已读,（美）贾雷德·戴蒙德著，谢延光译，上海译文出版社，2016.7,https://book.douban.com/subject/1813841/,枪炮、病菌与钢铁 : 人类社会的命运
2,历史,崩溃： 社会如何选择成败兴亡,None,（美）贾雷德·戴蒙德著，江滢、叶臻译，上海译文出版社，2018.3,https://book.douban.com/subject/27609290/,崩溃 : 社会如何选择成败兴亡
3,历史,贸易打造的世界： 1400年至今的社会、文化与世界经济,None,（美）彭慕兰、（美）史蒂夫·托皮克著，黄中宪、吴莉苇译，上海人民出版社，2018.2,https://book.douban.com/subject/27155983/,贸易打造的世界 : 1400年至今的社会、文化与世界经济
4,历史,伯罗奔尼撒战争史,None,（古希腊）修昔底德著，谢德风译，商务印书馆，2011.7,https://book.douban.com/subject/6893286/,伯罗奔尼撒战争史（上下）
...,...,...,...,...,...,...
195,文学,小说课,已读,许荣哲著，中信出版社，2016.8,https://book.douban.com/subject/26738798/,小说课（贰） : 偷故事的人
196,文学,文章自在,已读,张大春著，广西师范大学出版社，2017.1,https://book.douban.com/subject/26940987/,文章自在
197,文学,〈华尔街日报〉是如何讲故事的,在读,（美）威廉·E.布隆代尔著，徐扬译，华夏出版社，2006.1,None,None
198,文学,新新新闻主义： 美国顶尖非虚构作家写作技巧访谈录,None,（美）罗伯特·博因顿著，刘蒙之译，北京师范大学出版社，2018.2,https://book.douban.com/subject/30147440/,新新新闻主义 : 美国顶尖非虚构作家写作技巧访谈录


In [42]:
# 从现有excel表读取内容，只补充查询缺失链接
df_books = pd.read_excel(data_folder / '用200本书构建知识体系_tmp.xlsx')

df_part = df_books[['书名', '豆瓣链接', '备注']].fillna('')
results = []
for book_name, book_url, note in zip(df_part['书名'], df_part['豆瓣链接'], df_part['备注']):
    if book_url:
        results.append((book_url, note))
    else:
        book_name = book_name.strip('《》<>〈〉').split('：')[0]
        results.append(get_book_url(book_name, driver, False))
    
df_books['豆瓣链接'] = [i[0] for i in results]
df_books['备注'] = [i[1] for i in results]
df_books.to_excel(data_folder / '用200本书构建知识体系_tmp.xlsx', index=False)
df_books

已获取《宏调的逻辑》的豆瓣链接：[宏调的逻辑](https://book.douban.com/subject/26700300/)……
已获取《试错力》的豆瓣链接：[试错力](https://book.douban.com/subject/30131863/)……
已获取《华尔街日报〉是如何讲故事的》的豆瓣链接：[《华尔街日报》是如何讲故事的](https://book.douban.com/subject/1472946/)……
已获取《怎样讲好一个故事》的豆瓣链接：[哈佛非虚构写作课 : 怎样讲好一个故事](https://book.douban.com/subject/26662600/)……


,分类,书名,状态,说明,豆瓣链接,备注
0,历史,大历史： 虚无与万物之间,想读,（美）大卫·克里斯蒂安、（美）辛西娅·斯托克斯·布朗、（美）克雷格·本杰明著，刘耀辉译，北京...,https://book.douban.com/subject/26827546/,大历史 : 虚无与万物之间
1,历史,枪炮、病菌与钢铁： 人类社会的命运,已读,（美）贾雷德·戴蒙德著，谢延光译，上海译文出版社，2016.7,https://book.douban.com/subject/1813841/,枪炮、病菌与钢铁 : 人类社会的命运
2,历史,崩溃： 社会如何选择成败兴亡,NaN,（美）贾雷德·戴蒙德著，江滢、叶臻译，上海译文出版社，2018.3,https://book.douban.com/subject/27609290/,崩溃 : 社会如何选择成败兴亡
3,历史,贸易打造的世界： 1400年至今的社会、文化与世界经济,NaN,（美）彭慕兰、（美）史蒂夫·托皮克著，黄中宪、吴莉苇译，上海人民出版社，2018.2,https://book.douban.com/subject/27155983/,贸易打造的世界 : 1400年至今的社会、文化与世界经济
4,历史,伯罗奔尼撒战争史,NaN,（古希腊）修昔底德著，谢德风译，商务印书馆，2011.7,https://book.douban.com/subject/6893286/,伯罗奔尼撒战争史（上下）
...,...,...,...,...,...,...
195,文学,小说课,已读,许荣哲著，中信出版社，2016.8,https://book.douban.com/subject/26738798/,小说课（贰） : 偷故事的人
196,文学,文章自在,已读,张大春著，广西师范大学出版社，2017.1,https://book.douban.com/subject/26940987/,文章自在
197,文学,〈华尔街日报〉是如何讲故事的,在读,（美）威廉·E.布隆代尔著，徐扬译，华夏出版社，2006.1,https://book.douban.com/subject/1472946/,《华尔街日报》是如何讲故事的
198,文学,新新新闻主义： 美国顶尖非虚构作家写作技巧访谈录,NaN,（美）罗伯特·博因顿著，刘蒙之译，北京师范大学出版社，2018.2,https://book.douban.com/subject/30147440/,新新新闻主义 : 美国顶尖非虚构作家写作技巧访谈录


In [43]:
# 核对书名是否相符
for col in ['书名', '备注']:
    df_books[col] = df_books[col].str.replace(r'[<>《》:：、，, （）()〈〉]', '', regex=True)

df_books['核对'] = df_books.apply(lambda x: 'N' if x["书名"] != x["备注"] else 'Y', axis=1)

df_books.to_excel(data_folder / '用200本书构建知识体系_tm.xlsx', index=False)
df_books

,分类,书名,状态,说明,豆瓣链接,备注,核对
0,历史,大历史虚无与万物之间,想读,（美）大卫·克里斯蒂安、（美）辛西娅·斯托克斯·布朗、（美）克雷格·本杰明著，刘耀辉译，北京...,https://book.douban.com/subject/26827546/,大历史虚无与万物之间,Y
1,历史,枪炮病菌与钢铁人类社会的命运,已读,（美）贾雷德·戴蒙德著，谢延光译，上海译文出版社，2016.7,https://book.douban.com/subject/1813841/,枪炮病菌与钢铁人类社会的命运,Y
2,历史,崩溃社会如何选择成败兴亡,NaN,（美）贾雷德·戴蒙德著，江滢、叶臻译，上海译文出版社，2018.3,https://book.douban.com/subject/27609290/,崩溃社会如何选择成败兴亡,Y
3,历史,贸易打造的世界1400年至今的社会文化与世界经济,NaN,（美）彭慕兰、（美）史蒂夫·托皮克著，黄中宪、吴莉苇译，上海人民出版社，2018.2,https://book.douban.com/subject/27155983/,贸易打造的世界1400年至今的社会文化与世界经济,Y
4,历史,伯罗奔尼撒战争史,NaN,（古希腊）修昔底德著，谢德风译，商务印书馆，2011.7,https://book.douban.com/subject/6893286/,伯罗奔尼撒战争史上下,N
...,...,...,...,...,...,...,...
195,文学,小说课,已读,许荣哲著，中信出版社，2016.8,https://book.douban.com/subject/26738798/,小说课贰偷故事的人,N
196,文学,文章自在,已读,张大春著，广西师范大学出版社，2017.1,https://book.douban.com/subject/26940987/,文章自在,Y
197,文学,华尔街日报是如何讲故事的,在读,（美）威廉·E.布隆代尔著，徐扬译，华夏出版社，2006.1,https://book.douban.com/subject/1472946/,华尔街日报是如何讲故事的,Y
198,文学,新新新闻主义美国顶尖非虚构作家写作技巧访谈录,NaN,（美）罗伯特·博因顿著，刘蒙之译，北京师范大学出版社，2018.2,https://book.douban.com/subject/30147440/,新新新闻主义美国顶尖非虚构作家写作技巧访谈录,Y


In [ ]:
# 读取书单信息
book_list = data_folder / r'用200本书构建知识体系.xlsx'
df_books = pd.read_excel(book_list)

# 根据链接补充书名信息
df_part = df_books[['书名', '豆瓣链接', '备注']].fillna('')
results = []
for book_name, book_url, note in zip(df_part['书名'], df_part['豆瓣链接'], df_part['备注']):
    if note:
        results.append(note)
    else:
        driver.get(book_url)
        time.sleep(1)
        new_name = driver.find_elements(by=By.XPATH, value="//span[@property='v:itemreviewed']")[0].text
        results.append(new_name)
        print("{}->{}".format(book_name, new_name))
    
df_books['备注'] = results
df_books.to_excel(data_folder / '用200本书构建知识体系_tmp.xlsx', index=False)

## 创建豆瓣书单

In [ ]:
# 读取书单信息
book_list = data_folder / '用200本书构建知识体系.xlsx'
df_books = pd.read_excel(book_list)
df_books

In [4]:
df_books['分类'].unique()

array(['历史', '政治学、社会学、宏观经济学', '微观经济学、博弈论', '脑神经科学、遗传学、进化生物学、心理学、行为经济学',
       '科技创新、工程学', '逻辑学、统计学、数学、物理学、复杂科学、科学哲学', '哲学', '文学'], dtype=object)

In [88]:
# 获取已创建书单信息
driver.get('https://www.douban.com/people/2180307/subject_doulists/book')
tree = etree.HTML(driver.page_source)
existing_book_lists = tree.xpath("//ul[@class='doulist-list']/li/h3/a/text()")
print(f"已创建{len(existing_book_lists)}个书单：\n{existing_book_lists}。")

已创建5个书单：
['政治学、社会学、宏观经济学', '知识体系—科技创新、工程学', '知识体系—历史', '知识体系—大脑、遗传、进化、心理、行为经济', '知识体系—逻辑、统计、数学、物理、复杂科学']。


In [98]:
category = '哲学'
book_list_title = category
book_list_intro = "本书单来自何帆教授的《猜测和偏见》。200本推荐书籍中你读过多少本？\n\n> 创新会越来越多地来自边缘地带和交叉地带，所以我们必须把自己锻炼成一个终身学习者，不断构建一个属于自己的完整知识体系。\n> 选书的标准以晓畅、深刻为准。不选过于浅显的入门普及读物，也不选过于艰涩的专著。\n> 书目按八个领域划分，打破了传统的学科分类，这种划分方法仅代表个人的阅读习惯。"
all_book_lists_url = 'https://www.douban.com/people/2180307/subject_doulists/book'

# 截取某分类的书单信息
df_books_part = df_books.query("分类 == @category").dropna(subset=['豆瓣链接'])
book_names = df_books_part['书名']
book_urls = df_books_part['豆瓣链接']
# df_books_part

In [99]:
# 新建书单，若需验证码则手工操作
book_list_url = creat_book_list(driver, book_list_title, book_list_intro, all_book_lists_url)

In [100]:
# 若上一步创建书单失败或手工输入验证码，则须指定书单链接
book_list_url = 'https://www.douban.com/doulist/149649088/'

# 添加缺失书本到书单，注意让chrome页面在前台显示
existing_urls, count = add_books_to_list(driver, book_list_url, book_names, book_urls)
print(f"在书单中发现原有{len(existing_urls)}本书，新添加{count}本书。")

在书单中发现原有0本书，新添加25本书。


## 添加书籍到“想读”清单

In [6]:
def book_to_wishlist(book_url, book_name=None):
    """添加图书到“想读”"""
    driver.get(book_url)
    
    try:
        want_botton = driver.find_elements(by=By.XPATH, value="//input[@value='想读']")[0]
        want_botton.click()

        # 显式等待
        WebDriverWait(driver, 5, 0.5).until(EC.visibility_of_element_located((By.XPATH, "//input[@value='保存']")))

        save_botton = driver.find_elements(by=By.XPATH, value="//input[@value='保存']")[0]
        save_botton.click()
        if book_name:
            print(f"已添加《{book_name}》到“想读”清单。")
        time.sleep(2)
    except Exception as e:
        if book_name:
            print(f"添加《{book_name}》到“想读”清单失败！")
        

book_names = df_books['书名']
book_urls = df_books['豆瓣链接']

for book_name, book_url in zip(book_names, book_urls):
    book_to_wishlist(book_url, book_name)


添加《大历史： 虚无与万物之间》到“想读”清单失败！
添加《枪炮、病菌与钢铁： 人类社会的命运》到“想读”清单失败！
添加《崩溃： 社会如何选择成败兴亡》到“想读”清单失败！
添加《贸易打造的世界： 1400年至今的社会、文化与世界经济》到“想读”清单失败！
添加《伯罗奔尼撒战争史》到“想读”清单失败！
添加《罗马帝国的陨落： 一部新的历史》到“想读”清单失败！
添加《文明史： 人类五千年文明的传承与交流》到“想读”清单失败！
添加《西方的兴起： 人类共同体史》到“想读”清单失败！
添加《美国政治传统及其缔造者》到“想读”清单失败！
添加《现代世界的诞生》到“想读”清单失败！
添加《论美国的民主》到“想读”清单失败！
添加《旧制度与大革命》到“想读”清单失败！
添加《战争史》到“想读”清单失败！
添加《革命的年代》到“想读”清单失败！
添加《梦游者： 1914年，欧洲如何走向“一战”》到“想读”清单失败！
添加《凯恩斯传》到“想读”清单失败！
添加《大转型： 我们时代的政治与经济起源》到“想读”清单失败！
添加《国富国穷》到“想读”清单失败！
添加《大分流： 欧洲、中国及现代世界经济的发展》到“想读”清单失败！
添加《中国历代政治得失》到“想读”清单失败！
添加《万历十五年》到“想读”清单失败！
添加《中国史通论》到“想读”清单失败！
添加《大象的退却： 一部中国环境史》到“想读”清单失败！
添加《王氏之死： 大历史背后的小人物命运》到“想读”清单失败！
添加《叫魂： 1768年中国妖术大恐慌》到“想读”清单失败！
添加《变化社会中的政治秩序》到“想读”清单失败！
添加《文明的冲突与世界秩序的重建》到“想读”清单失败！
添加《政治秩序的起源： 从前人类时代到法国大革命》到“想读”清单失败！
添加《政治秩序与政治衰败： 从工业革命到民主全球化》到“想读”清单失败！
添加《独裁者手册》到“想读”清单失败！
添加《大国政治的悲剧》到“想读”清单失败！
添加《信号与欺骗： 国际关系中的形象逻辑》到“想读”清单失败！
添加《即将到来的地缘战争： 无法回避的大国冲突及对地理宿命的抗争》到“想读”清单失败！
添加《硬球： 政治是这样玩的》到“想读”清单失败！
添加《艾希曼在耶路撒冷： 一份关于平庸的恶的报告》到“想读”清单失败！
添加《公正》到“想读”清单失败！
添加《反潮流： 